In [153]:
import time
import plotly.graph_objects as go

In [154]:
class Vertex:
    def __init__(self,x_coord, y_coord,next_index, prev_index, index):
        self.x = x_coord
        self.y = y_coord
        self.next = next_index
        self.prev = prev_index
        self.index = index


In [155]:
class Polygon:
    def __init__(self, filename):
        self.x_coords = []
        self.y_coords = []
        self.vertex_list = []
        with open(filename, 'r') as file:
            for line in file:
                parts = line.strip().split()
                num_vertices = int(parts[0])
                #print(num_vertices)
                for i in range(num_vertices):
                    x_str, y_str = parts[2 * i + 1], parts[2 * i + 2]
                    num, dem = map(int, x_str.split('/'))
                    x_coord = num / dem
                    self.x_coords.append(x_coord)
                    num, dem = map(int, y_str.split('/'))
                    y_coord = num / dem
                    self.y_coords.append(y_coord)
                    self.vertex_list.append(Vertex(x_coord, y_coord, (i+1)%num_vertices, (i-1)%num_vertices , i))
    
    def plot(self, triangles=None):
        fig = go.Figure()
    
        fig.add_trace(go.Scatter(x=self.x_coords + [self.x_coords[0]], y=self.y_coords + [self.y_coords[0]], fill='toself', fillcolor='rgba(0, 0, 139, 0.2)',
                                 mode='lines+markers', name='Triangle'))
    
    
        fig.update_layout(
            xaxis=dict(title='X'),
            yaxis=dict(title='Y'),
            showlegend=True,
            hovermode='closest',
            template='plotly_white',
            updatemenus=[{'type': 'buttons',
                          'buttons': [{'label': 'Play',
                                       'method': 'animate',
                                       'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}]},
                                     {'label': 'Pause',
                                      'method': 'animate',
                                      'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate', 'transition': {'duration': 0}}]}]}]
        )
        if triangles:
            frames = []
            fig.add_trace(go.Scatter(x=self.x_coords + [self.x_coords[0]], y=self.y_coords + [self.y_coords[0]], fill='toself', fillcolor='rgba(34, 200, 255, 0.2)',
                                     mode='lines+markers', name='Polygon'))

            triangles_x = []
            triangles_y = []

            for i, triangle in enumerate(triangles):
                triangle_x = [triangle[0].x, triangle[1].x, triangle[2].x, triangle[0].x]
                triangle_y = [triangle[0].y, triangle[1].y, triangle[2].y, triangle[0].y]


                triangles_x.extend(triangle_x + [None])
                triangles_y.extend(triangle_y + [None])

                frame = go.Frame(data=[go.Scatter(x=triangles_x, y=triangles_y, mode='lines', line=dict(color='blue'), name='Triangles')],
                                 name=f'frame{i}')
                frames.append(frame)

            fig.frames = frames
    
        fig.show()

## Funçõs para tringulação


In [156]:

def triangulate(poly):
        triangles = []
        vertices = poly.vertex_list[:]
        
        while len(vertices) > 3:
            for i in range(len(vertices)):
                prev_index = (i - 1) % len(vertices)
                next_index = (i + 1) % len(vertices)
                
                ear_found = ear(vertices[prev_index], vertices[i], vertices[next_index],vertices)
                if ear_found:
                    triangles.append((vertices[prev_index], vertices[i], vertices[next_index]))
                    #self.plot(triangles)
                    vertices.pop(i)
                    break
        
        triangles.append((vertices[0], vertices[1], vertices[2]))

        return triangles

# Ear Clipping


In [157]:
def ear(prev_vertex, ear_vertex, next_vertex, vertices):
        if convex(prev_vertex, ear_vertex, next_vertex):
            for v in vertices:
                if v != prev_vertex and v != ear_vertex and v != next_vertex:
                    if inTriangle(v, prev_vertex, ear_vertex, next_vertex):
                        return False
            return True
        return False
    
def convex(prev_vertex, curr_vertex, next_vertex):
    return vetorialProduct(prev_vertex, curr_vertex, next_vertex) > 0

def vetorialProduct(v1, v2, v3):
    return (v2.x - v1.x) * (v3.y - v1.y) - (v2.y - v1.y) * (v3.x - v1.x)


def inTriangle(v, v1, v2, v3):
    d1 = vetorialProduct(v2,v,v1)
    d2 = vetorialProduct(v3,v,v2)
    d3 = vetorialProduct(v1,v,v3)
    return not ((d1 < 0 or d2 < 0 or d3 < 0) and (d1 > 0 or d2 > 0 or d3 > 0))


In [158]:
input = 'agp2009a-simplerand/randsimple-20-1.pol'
poly = Polygon(input)
t_i = time.time()
triangulate = triangulate(poly)
t_f = time.time()
poly.plot(triangulate)
print(f"tempo levado: {(t_f-t_i):.4f}")
print(f"Número de triangulos: {len(triangulate)}")

tempo levado: 0.0003
Número de triangulos: 18


Referências:

https://www.geeksforgeeks.org/timing-functions-with-decorators-python/

https://www.geometrictools.com/Documentation/TriangulationByEarClipping.pdf

https://github.com/yaugenst/triangulation/tree/master

https://www.w3schools.com/css/css_colors_rgb.asp